In [1]:
import numpy as np
import random
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import import_ipynb
import CFAR_v2 as cfar
import GeoProcess as gp

importing Jupyter notebook from CFAR_v2.ipynb
importing Jupyter notebook from GeoProcess.ipynb


In [2]:
band_arr = gp.readGeoTiff('data/testImage.tif')
# subset_arr = band_arr[808:870,2202:2264]
# subset_arr.shape
# gp.visualizeImg(subset_arr)
# gp.save_img2Geotiff(subset_arr,'data/testImage.tif')

In [3]:
class GeneticAlgorithm(object):
    
    def __init__(self,chromosome_length,population_size,no_of_iteration):
        
        self.population_size = population_size
        self.chromosome_length = chromosome_length
        self.no_of_iteration = no_of_iteration
        self.bestresult_value = 0.0
        self.bestresult_key = 0.0
        print("Populating the Training set.")
        self.population = ([[random.randint(0,1) for x in range(self.chromosome_length)] 
                            for i in tqdm(range(self.population_size))])
        self.fitnessScore = []
        self.parents = []
        Shpdetc = cfar.CFAR_v2(band_arr,50,52,54,1)
        self.dvi,self.threshold = Shpdetc.computeDVnThreshold()
        print("Kernel Ready.")
    
    def shipDetection(self,chromosome):
        f_img = []       
        DV = self.dvi
        T = (chromosome*self.threshold).reshape(band_arr.shape)
        #T = self.pfa
        
        for i in range(band_arr.shape[0]):
            for j in range(band_arr.shape[1]):
                
                if DV[i][j] > T[i][j]:
                    
                    f_img.append(1)
                else:
                    f_img.append(0) #valid Ships
        
        f_img = np.array(f_img).reshape(band_arr.shape)
        #print("Binary Image of Ships is Succesfully Generated.\n")
        return f_img
    
    def fitness_function(self):
        fit_value = []
        for i in range(self.population_size):
            chromosome_value=0
            for j in range(self.chromosome_length-1,-1,-1):
                #print(i,j)
                chromosome_value += self.population[i][j]*(2**(0-j))
                
            #chromosome_value = -1*chromosome_value if self.population[i][0]==1 else chromosome_value
            #print(chromosome_value)
            
            final_img = self.shipDetection(chromosome_value)         
            no_of_pixel = self.countShipPixels(final_img)
            #print(no_of_pixel)
            #fit_value.append(abs((chromosome_value**2) - 6.25 ))
            cutoff = 150-no_of_pixel
            if cutoff < 0 and cutoff > -10 :
                fit_value.append(abs(cutoff))
            else:
                fit_value.append(150)
            #print(fit_value)
        fit_value, self.population = zip(*sorted(zip(fit_value, self.population) , reverse = True))
        self.bestresult_value = fit_value[self.population_size-1]
        self.bestresult_key = self.population[self.population_size-1]
        self.fitnessScore.append(self.bestresult_value)
        
    def selectParents(self):
        self.parents= self.population[int(self.population_size/2):]
        
    def crossOver(self):
        #cross_point = random.randint(0,self.chromosome_length)
        cross_point = 2
        upto = len(self.parents)
        #print(len(self.parents))
        for i in range(0,upto,2):
            #print(i)
            self.parents= self.parents + tuple([(self.parents[i][0:cross_point +1] + self.parents[i+1]
                                      [cross_point+1:self.chromosome_length])])
            self.parents = self.parents + tuple([(self.parents[i+1][0:cross_point +1] + self.parents[i]
                                      [cross_point+1:self.chromosome_length])])
    
    def mutation(self) :
        #print(self.parents)
        mute = random.randint(0,100)
        if mute %2 == 0 :
            x = random.randint(0,self.population_size-1)
            y = random.randint(0,self.chromosome_length-1)
            #print(x,y)
            
            self.parents[x][y] = 1-self.parents[x][y]
        self.populations = self.parents
        
    def executeGA(self):
        for i in tqdm(range(self.no_of_iteration)) :
            print("Processing Generation: ",i)
            self.fitness_function()
            self.selectParents()
            self.crossOver()
            self.mutation()
            print("Optimised Value Obtained : "+str(self.bestresult_value)+"\n")
        print("Sucessfully found the best optimized results.")
        
    def getResults(self,plot=False):
        chromosome_value = 0.0
        print("The Optimized value obtained: ",self.bestresult_value)
        for i in range((self.chromosome_length)-1,-1,-1):
            chromosome_value += self.bestresult_key[i]*(2**(0-i))
        print("The Optimized result is found at: ",chromosome_value)
        
        if plot:
            plt.plot([i for i in range(len(self.fitnessScore))],self.fitnessScore)
            plt.xlabel('Generation')
            plt.ylabel('Fitness Score')
            plt.title('GA Analysis')
    
    def countShipPixels(self,final_img):
        count = 0
        for i in range(final_img.shape[0]):
            for j in range(final_img.shape[1]):
                if final_img[i,j] == 1:
                    count += 1
        return count

In [15]:
# ga = GeneticAlgorithm(chromosome_length=9,population_size=1000,no_of_iteration=1000)
# ga.executeGA()
# ga.getResults(plot=True)